In [2]:
import subprocess

# Startet `ollama serve` im Hintergrund
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Ollama server is now running in the background.") 

Ollama server is now running in the background.


In [ ]:
# Lege das LLM fest, mit dem nachfolgend gearbeitet wird.
model = "qwen2.5:7b"

In [4]:
# Gewähltes Modell von Ollama herunterladen
process = subprocess.Popen(["ollama", "pull", model], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ausgabe anzeigen
stdout, stderr = process.communicate()
print(stdout.decode("utf-8"))

if stderr:
    print("Error:", stderr.decode("utf-8"))
else:
    print(f"{model} pulled successfully.")


Error: pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 2bad

In [ ]:
import faker
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
import os

# Seed setzen für Reproduzierbarkeit
random.seed(42)
Faker.seed(42)
fake = Faker()

# Ordner 'tables' erstellen, falls nicht vorhanden
output_folder = "tables"
os.makedirs(output_folder, exist_ok=True)

# Funktion, um zufällige Datumswerte zu erzeugen
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Anzahl der Datensätze
num_students = 100
num_courses = 10
num_professors = 5
num_departments = 3
num_enrollments = 300

# 1. STUDENT_DIMENSION - Generierung der Studentendaten
student_data = {
    "Student_ID": range(1, num_students + 1),
    "First_Name": [fake.first_name() for _ in range(num_students)],
    "Last_Name": [fake.last_name() for _ in range(num_students)],
    "Date_of_Birth": [random_date(datetime(1990, 1, 1), datetime(2002, 12, 31)).date() for _ in range(num_students)],
    "Enrollment_Date": [random_date(datetime(2020, 1, 1), datetime(2023, 12, 31)).date() for _ in range(num_students)],
    "Email": [f"{fake.first_name().lower()}.{fake.last_name().lower()}@example.com" for _ in range(num_students)]
}
student_df = pd.DataFrame(student_data)

# 2. COURSE_DIMENSION - Generierung der Kursdaten
course_names = [
    "Data Science Basics", "Advanced Machine Learning", "Database Systems",
    "Statistics for Data Science", "Programming with Python", "Ethics in AI",
    "Big Data Analysis", "Data Visualization", "Project Management", "Deep Learning"
]
course_data = {
    "Course_ID": range(1, num_courses + 1),
    "Course_Name": course_names,
    "Credits": [random.choice([3, 4, 5]) for _ in range(num_courses)],
    "Department_ID": [random.randint(1, num_departments) for _ in range(num_courses)]
}
course_df = pd.DataFrame(course_data)

# 3. PROFESSOR_DIMENSION - Generierung der Professorendaten
professor_data = {
    "Professor_ID": range(1, num_professors + 1),
    "First_Name": [fake.first_name() for _ in range(num_professors)],
    "Last_Name": [fake.last_name() for _ in range(num_professors)],
    "Email": [f"{fake.first_name().lower()}.{fake.last_name().lower()}@university.com" for _ in range(num_professors)]
}
professor_df = pd.DataFrame(professor_data)

# 4. DEPARTMENT_DIMENSION - Generierung der Abteilungsdaten
department_names = ["Computer Science", "Business Administration", "Psychology"]
department_data = {
    "Department_ID": range(1, num_departments + 1),
    "Department_Name": department_names
}
department_df = pd.DataFrame(department_data)

# 5. ENROLLMENT_FACTS - Generierung der Einschreibungen
enrollment_data = {
    "Enrollment_ID": range(1, num_enrollments + 1),
    "Student_ID": [random.randint(1, num_students) for _ in range(num_enrollments)],
    "Course_ID": [random.randint(1, num_courses) for _ in range(num_enrollments)],
    "Professor_ID": [random.randint(1, num_professors) for _ in range(num_enrollments)],
    "Enrollment_Date": [random_date(datetime(2021, 1, 1), datetime(2023, 12, 31)).date() for _ in range(num_enrollments)],
    "Grade": [random.choice(['A', 'B', 'C', 'D', 'F']) for _ in range(num_enrollments)]
}
enrollment_df = pd.DataFrame(enrollment_data)

# Speichern der DataFrames als CSV-Dateien im 'tables'-Ordner
student_df.to_csv(os.path.join(output_folder, "STUDENT_DIMENSION.csv"), index=False)
course_df.to_csv(os.path.join(output_folder, "COURSE_DIMENSION.csv"), index=False)
professor_df.to_csv(os.path.join(output_folder, "PROFESSOR_DIMENSION.csv"), index=False)
department_df.to_csv(os.path.join(output_folder, "DEPARTMENT_DIMENSION.csv"), index=False)
enrollment_df.to_csv(os.path.join(output_folder, "ENROLLMENT_FACTS.csv"), index=False)

print("CSV-Dateien wurden erfolgreich im 'tables'-Ordner gespeichert.")


CSV-Dateien wurden erfolgreich im 'tables'-Ordner gespeichert.
